In [1]:
from renju import run, run_test, Player
from agent import BackendAgent

import collections
import itertools
import logging
import time

%matplotlib notebook

In [2]:
DEF_PY_PATH = '/home/dasimagin/sandbox/renju/src'

class Config:
    def __init__(self, name, cmd, cwd, py_path=DEF_PY_PATH):
        self.name = name
        self.cmd = cmd
        self.cwd = cwd
        self.py_path = py_path
        
    def create_agent(self):
        return BackendAgent(
            self.cmd,
            self.name,
            env = {'PYTHONPATH': self.py_path},
            cwd = self.cwd
        )
        
hardy = Config(
    'Hardy',
    '/home/dasimagin/sandbox/envs/hardy/bin/python /home/dasimagin/sandbox/hardy/renju/search_agent.py',
    '/home/dasimagin/sandbox/hardy/renju/'
)

pierre = Config(
    'Pierre',
    '/home/dasimagin/sandbox/envs/pierre/bin/python /home/dasimagin/sandbox/pierre/contest/tree_search.py',
    '/home/dasimagin/sandbox/pierre/contest'
)

lolita = Config(
    'Lolita',
    '/home/dasimagin/sandbox/envs/lolita/bin/python /home/dasimagin/sandbox/lolita/contest/my_agent.py',
    '/home/dasimagin/sandbox/lolita/contest'
)

elena = Config(
    'Elena',
    '/home/dasimagin/sandbox/envs/elena/bin/python /home/dasimagin/sandbox/elena/src/ham_agent.py',
    '/home/dasimagin/sandbox/elena/src'
)

john = Config(
    'John 8=э',
    '/home/dasimagin/sandbox/envs/john/bin/python /home/dasimagin/sandbox/john/competition/player/mcts_agent.py',
    '/home/dasimagin/sandbox/john/competition/player'
)

configs = [hardy, pierre, lolita, elena, john]

In [3]:
def competition(configs, game_n=3, max_move_n=60, timeout=10, timeinit=20):
    table = {config.name: 0 for config in configs}
    logging.debug('Start comptetion...')
    
    for pair in itertools.combinations(configs, 2):
        for n in range(2 * game_n):
            black = pair[n % 2].create_agent() 
            white = pair[(n + 1) % 2].create_agent()                
                
            logging.debug('Load backends...')
            time.sleep(timeinit)
            
            logging.debug('Start game...')
            result, moves = run(black, white, max_move_n=60, timeout=10)
            logging.debug(
                '{black} vs {white}: {moves} {result}'.format(
                    black = black.name(),
                    white = white.name(),
                    moves = moves,
                    result = str(result)
                )
            )
            
            if result == Player.BLACK:
                table[black.name()] += 3
                
            if result == Player.WHITE:
                table[white.name()] += 3
                
            if not result:
                table[black.name()] += 1
                table[white.name()] += 1
                
            logging.debug('Current results {}'.format(table))
                
    return table

In [4]:
logging.basicConfig(
    format='%(asctime)s %(message)s',
    filename='competition.log',
    level=logging.DEBUG
)

table = competition(configs, game_n=1)

In [ ]:
for name, score in table:
    print('{name}: {score}'.format(name=name, score=score))